## 탈잉 
### 카테고리 :취미/공예 

- 총 125 페이지
- 페이지 당 15개 클래스
- likes,link,location,number,price,teacher,title
- 'location'의 경우 타 사이트와의 비교 후 tile과 합칠 예정 `'[' + location + ']' + title`

- 해당 카테고리 내에 있는 1873개(오후기준: 1875개) 클래스 중 모두 크롤링 완료

In [3]:
import scrapy, requests, re 
from scrapy.http import TextResponse

import time
import pandas as pd

In [2]:
# 프로젝트 생성

In [2]:
!scrapy startproject tal_ing

New Scrapy project 'tal_ing', using template directory 'C:\Users\study\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\study\Desktop\crawling\tal_ing

You can start your first spider with:
    cd tal_ing
    scrapy genspider example example.com


In [4]:
# link

In [3]:
req = requests.get("https://taling.me/Home/Search/?cateMain=22")
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://taling.me/Home/Search/?cateMain=22>

In [10]:
# xpath 찾기 (page 1)

In [4]:
links = response.xpath('//*[@id="top-space"]/div/div/a/@href').extract()
len(links), links

(15,
 ['/Talent/Detail/33374',
  '/Talent/Detail/32152',
  '/Talent/Detail/3750',
  '/Talent/Detail/18571',
  '/Talent/Detail/3281',
  '/Talent/Detail/7060',
  '/Talent/Detail/12005',
  '/Talent/Detail/15043',
  '/Talent/Detail/15175',
  '/Talent/Detail/3309',
  '/Talent/Detail/4109',
  '/Talent/Detail/4617',
  '/Talent/Detail/28671',
  '/Talent/Detail/9057',
  '/Talent/Detail/3922'])

In [5]:
links = list(map(response.urljoin, links))
links

['https://taling.me/Talent/Detail/33374',
 'https://taling.me/Talent/Detail/32152',
 'https://taling.me/Talent/Detail/3750',
 'https://taling.me/Talent/Detail/18571',
 'https://taling.me/Talent/Detail/3281',
 'https://taling.me/Talent/Detail/7060',
 'https://taling.me/Talent/Detail/12005',
 'https://taling.me/Talent/Detail/15043',
 'https://taling.me/Talent/Detail/15175',
 'https://taling.me/Talent/Detail/3309',
 'https://taling.me/Talent/Detail/4109',
 'https://taling.me/Talent/Detail/4617',
 'https://taling.me/Talent/Detail/28671',
 'https://taling.me/Talent/Detail/9057',
 'https://taling.me/Talent/Detail/3922']

In [9]:
# 강좌정보 xpath

In [6]:
link = links[5]
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://taling.me/Talent/Detail/7060>

In [14]:
site = '탈잉'
title = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/h1/text()')[0].extract()
teacher = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/em/text()')[0].extract()
number = response.xpath('//*[@id="wrap"]/div[2]/aside/div/p/span[2]/span/text()')[0].extract()
price = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/b/text()')[0].extract()
contentment = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/span/i[2]/text()')[0].extract()
location = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/div/p[2]/text()')[0].extract().strip()
soldout = response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text').extract()

site, title, teacher, number, s_price, contentment, location, soldout

('요즘 대세는 쥬브썸 플라워 스튜디오 :)',
 '신지우 튜터',
 '총 1회 2시간',
 '40,000원 /',
 '5.0',
 '신촌홍대',
 [])

In [ ]:
# items.py 

In [8]:
%%writefile tal_ing/tal_ing/items.py
import scrapy


class TalIngItem(scrapy.Item):
    site = scrapy.Field()
    title = scrapy.Field()
    teacher = scrapy.Field()
    number = scrapy.Field()
    price = scrapy.Field()
    contentment = scrapy.Field()
    link = scrapy.Field()
    soldout = scrapy.Field()
    

Overwriting tal_ing/tal_ing/items.py


In [28]:
# spider.py

In [9]:
%%writefile talinghobby/talinghobby/spiders/spider.py
import scrapy
from tal_ing.items import TalIngItem


class TalIngSpider(scrapy.Spider):
    name = "TalingHobby"
    allow_domain = ["taling.me"]
    pages = range(1, 125+1)
    start_urls = [f"https://taling.me/Home/Search/?page={page}&cateMain=22&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=" for page in pages]
    
    def parse(self, response):
        links = response.xpath('//*[@id="top-space"]/div/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            #time.sleep(120) 
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = TalIngItem()
        item['site'] = '탈잉'
        item['category1'] = '취미/공예'
        item['category2'] = 'nan'
        location = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/div/p[2]/text()')[0].extract().strip()
        item["title"] = '[' + location + ']' + response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/h1/text()')[0].extract()
        item["teacher"] = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/em/text()')[0].extract()
        try:
            item["number"] = response.xpath('//*[@id="wrap"]/div[2]/aside/div/p/span[2]/span/text()')[0].extract()
        except:
            item["number"] = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/span/text()')[0].extract()
        item["price"] = '총 ' + response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/b/text()')[0].extract().replace(' /', '')
        try:
            item["likes"] = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/span/i[2]/text()')[0].extract()
        except:
            item["likes"] = "(0)"
        soldout = response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text()').extract()
        if soldout not in _:
            pass
        else:
            item["soldout"] = response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text()').extract()
        
        item["link"] = response.url
        #time.sleep(120)
        yield item
    
    

Overwriting tal_ing/tal_ing/spiders/spider.py


In [8]:
pages = range(1, 125+1)
start_urls = [f"https://taling.me/Home/Search/?page={page}&cateMain=22&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=" for page in pages]
req = requests.get(start_urls[0])
response = TextResponse(req.url, body=req.text, encoding="utf-8")

links = response.xpath('//*[@id="top-space"]/div/div/a/@href').extract()
links = list(map(response.urljoin, links))
req = requests.get(links[1])
response = TextResponse(req.url, body=req.text, encoding="utf-8")

site = '탈잉'
location = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/div/p[2]/text()')[0].extract().strip()

title = '[' + location + ']' + response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/h1/text()')[0].extract()
teacher = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/em/text()')[0].extract()
number = response.xpath('//*[@id="wrap"]/div[2]/aside/div/p/span[2]/span/text()')[0].extract()
s_price = '총 ' + response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/b/text()')[0].extract().replace(' /', '')
contentment = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/span/i[2]/text()')[0].extract() 
#soldout= response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text()').extract()
link = response.url


site, title, teacher, number, s_price, contentment

('탈잉',
 '[신촌홍대]요즘 대세는 쥬브썸 플라워 스튜디오 :)',
 '신지우 튜터',
 '총 1회 2시간',
 '총 40,000원',
 '5.0')

In [2]:
%pwd

'C:\\Users\\study\\Desktop\\git\\Tal_ing'

In [32]:
%ls tal_ing

 Volume in drive C has no label.
 Volume Serial Number is 7DCE-F2C1

 Directory of C:\Users\study\Desktop\crawling\tal_ing\tal_ing

2021-03-02  �삤�썑 07:11    <DIR>          .
2021-03-02  �삤�썑 07:11    <DIR>          ..
2021-02-23  �삤�썑 08:28                 0 __init__.py
2021-03-02  �삤�썑 07:11    <DIR>          __pycache__
2021-03-02  �삤�썑 06:48               258 items.py
2021-03-02  �삤�썑 06:48             3,648 middlewares.py
2021-03-02  �삤�썑 06:48               360 pipelines.py
2021-03-02  �삤�썑 06:48             3,065 settings.py
2021-03-02  �삤�썑 07:11    <DIR>          spiders
               5 File(s)          7,331 bytes
               4 Dir(s)  343,526,658,048 bytes free


In [11]:
%%writefile run.sh
cd C:\Users\study\Desktop\crawling\tal_ing
scrapy crawl TalingHobby -o taling-test5.csv

Overwriting run.sh


In [27]:
!ls

scrapy.cfg
tal_ing


In [61]:
%cd tal_ing/

C:\Users\study\Desktop\crawling\tal_ing


In [36]:
!pwd

/c/Users/study/Desktop/crawling/tal_ing


In [26]:
!/bin/bash run.sh         

The system cannot find the path specified.


In [28]:
#!scrapy crawl TalingHobby -o taling-test5.csv -t csv

^C


In [12]:
pd.read_csv("tal_ing/taling-hobby.csv")

,likes,link,location,number,price,teacher,title
0,4.9,https://taling.me/Talent/Detail/15128,강남,총 1회 2시간,"30,000원 /",박현준 튜터,"[원데이] 나만의 느낌있는 드로잉으로, 나만의 드로잉 하기!"
1,5.0,https://taling.me/Talent/Detail/23609,강서,총 1회 2시간,"78,000원 /",KIKO 튜터,샹들리에 케이크: 하루 배워 해마다 써먹기! (앙금+생크림 조합)
2,5.0,https://taling.me/Talent/Detail/31172,왕십리,총 4회 4시간,"150,000원 /",SCHOLAR 튜터,[연습생 출신/현직 탑라이너/작곡가] 자신에게 맞는 커리큘럼으로 개성과 실력을 동시에!
3,5.0,https://taling.me/Talent/Detail/3503,강남 온라인 Live 논현,총 1회 2시간,"50,000원 /",남재현 튜터,[강남 1위/90분 원데이] '최고의 장비' 분석을 통해 '고음' '발성 문제' 바...
4,5.0,https://taling.me/Talent/Detail/20164,구로,총 4회 4시간,"150,000원 /",신성우 튜터,(작곡클래스) 방구석 건반/작곡으로 끝장 보자!! [구로디지털단지][신림]
...,...,...,...,...,...,...,...
1870,(0),https://taling.me/Talent/Detail/13311,대구중앙,총 1회 2시간,"20,000원 /",김미선 튜터,인스타사진 예쁘게 찍는 꿀팁/촬영/보정/ 필름카메라로 인생샷 남기기까지 단 하루
1871,(0),https://taling.me/Talent/Detail/8358,구월동 수원,총 1회 3시간,"84,000원 /",임은영 튜터,[인천/원데이] 특별한 날 특별한 분께 드리는 플라워 앙금케이크
1872,(0),https://taling.me/Talent/Detail/7426,서현 잠실,총 5회 10시간,"120,000원 /",김혜진 튜터,"(발손가능) 여행,영화의 특별한 순간들을 나만의 일러스트로 남기기!"
1873,(0),https://taling.me/Talent/Detail/25979,신촌홍대,총 1회 3시간,"90,000원 /",장세미 튜터,레터링 케이크 만들기 실전반 클래스
